# BERT model 

In this notebook, we will train a system based on BERT to classify the sentiment of the tweets about Covid-19. 


In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')
root='drive/My Drive/Colab Notebooks/seminarioUPM/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[BERT - Kaggle](https://www.kaggle.com/nayansakhiya/text-classification-using-bert)

In [50]:
import pandas as pd
import numpy as np
# reading the already cleaned data, paths are those obtained from google collab
train_data = pd.read_csv(root+'data/train.csv')
test_data = pd.read_csv(root+'data/test.csv')
print(f'Training instances: {train_data.shape[0]}, and testing instances: {test_data.shape[0]}')
# dropping indexes with nan

# training set
index_with_nan = train_data.index[train_data.isnull().any(axis=1)]
train_data.drop(index_with_nan,0, inplace=True)

# testing set
index_with_nan = test_data.index[test_data.isnull().any(axis=1)]
test_data.drop(index_with_nan,0, inplace=True)

#texts
X_train = train_data.iloc[:,0]
X_test = test_data.iloc[:,0]
#labels
y_train = train_data.iloc[:,1]
y_test = test_data.iloc[:,1]

#from categorial to one-hot
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

# making the  data into numpy formats
train_text=X_train.to_numpy()
train_labels=y_train.to_numpy()
#train_labels = np.asarray(train_labels).astype('float32')

#train_labels=np.array(train_labels, dtype=np.float)


test_text = X_test.to_numpy()
test_labels = y_test.to_numpy()

Training instances: 41156, and testing instances: 3798


In [51]:
#BERT
from transformers import BertTokenizer
TOKENIZER = BertTokenizer.from_pretrained("bert-base-uncased")
enc = TOKENIZER.encode("Encode me!")
dec = TOKENIZER.decode(enc)
print("Encode: " + str(enc))
print("Decode: " + str(dec))

Encode: [101, 4372, 16044, 2033, 999, 102]
Decode: [CLS] encode me! [SEP]


In [52]:
import tensorflow as tf
from transformers import TFBertModel, BertModel
#get BERT layer
bert_base = TFBertModel.from_pretrained('bert-base-uncased')
#bert_base = BertModel.from_pretrained('bert-base-uncased')          #to use with PyTorch

#select BERT tokenizer
TOKENIZER = BertTokenizer.from_pretrained("bert-base-uncased")

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [74]:
import pyarrow as pa

def bert_encode(texts,maximum_len) :
    input_ids = []
    attention_masks = []
  

    for i in range(len(texts)):
        encoded = TOKENIZER.encode_plus(texts[i],
                                        add_special_tokens=True,
                                        max_length=maximum_len,
                                        #pad_to_max_length=True,
                                        truncation=True,
                                        return_attention_mask=True,
                                        )
      
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        
    return np.array(input_ids),np.array(attention_masks)


In [78]:

#train BERT base model? 
MAX_SEQUENCE_LENGTH=30

#get our inputs
print('Encoding Tweets...')
train_input_ids,train_attention_masks = bert_encode(train_text,MAX_SEQUENCE_LENGTH)
test_input_ids,test_attention_masks = bert_encode(test_text,MAX_SEQUENCE_LENGTH)
print('Tweets encoded')
print('')

#debugging step
print('Train length:', len(train_input_ids))
print('Test length:', len(test_input_ids))


Encoding Tweets...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Tweets encoded

Train length: 41087
Test length: 3795


In [79]:
ADD_DENSE=False
ADD_DROPOUT=False
DROPOUT=0.2
DENSE_DIM=64
NUMCLASSES=5

@tf.autograph.experimental.do_not_convert
def build_model(model_layer, learning_rate=0.001, add_dense = ADD_DENSE,
               dense_dim = DENSE_DIM, add_dropout = ADD_DROPOUT, dropout = DROPOUT, numclasses=NUMCLASSES):
    
    #define inputs
    input_ids = tf.keras.Input(shape=(MAX_SEQUENCE_LENGTH,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(MAX_SEQUENCE_LENGTH,),dtype='int32')
    
    #insert BERT layer
    transformer_layer = model_layer([input_ids,attention_masks])
    
    #choose only last hidden-state
    output = transformer_layer[1]
    
    
    #add dense relu layer
    if add_dense:
        print("Training with additional dense layer...")
        output = tf.keras.layers.Dense(dense_dim,activation='relu')(output)
    
    #add dropout
    if add_dropout:
        print("Training with dropout...")
        output = tf.keras.layers.Dropout(dropout)(output)
    
    output = tf.keras.layers.Dense(numclasses,activation='softmax')(output)
    
    #assemble and compile
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)

    model.compile(tf.keras.optimizers.Adam(lr=learning_rate), loss='CategoricalCrossentropy', metrics=['accuracy'])
    
    return model

In [80]:
#and build and view parameters
BERT_base = build_model(bert_base, learning_rate = 1e-5)
BERT_base.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
tf_bert_model_4 (TFBertModel)   multiple             109482240   input_11[0][0]                   
                                                                 input_12[0][0]                   
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 5)            3845        tf_bert_model_4[2][1]      

In [81]:

EPOCHS=2
BATCH_SIZE=16

checkpoint = tf.keras.callbacks.ModelCheckpoint('base_model.h5', monitor='val_loss', save_best_only = True, save_weights_only = True)
history = BERT_base.fit([train_input_ids,train_attention_masks], 
                        train_labels, 
                        validation_split = 0.2, 
                        epochs = EPOCHS, 
                        callbacks = [checkpoint], 
                        batch_size = BATCH_SIZE)   

ValueError: ignored

In [21]:
#define conveient training function to visualize learning curves
def plot_learning_curves(history): 
    fig, ax = plt.subplots(1, 2, figsize = (20, 10))

    ax[0].plot(history.history['accuracy'], color = '#171820')
    ax[0].plot(history.history['val_accuracy'], color = '#fdc029')

    ax[1].plot(history.history['loss'], color='#171820')
    ax[1].plot(history.history['val_loss'], color = '#fdc029')

    ax[0].legend(['train', 'validation'], loc = 'upper left')
    ax[1].legend(['train', 'validation'], loc = 'upper left')

    fig.suptitle("Model Learning Curves", fontsize=14)

    ax[0].set_ylabel('Accuracy')
    ax[0].set_xlabel('Epoch')
    ax[1].set_ylabel('Loss')
    ax[1].set_xlabel('Epoch')

    return plt.show()


In [ ]:
plot_learning_curves(history)

EVALUATION


In [ ]:
preds_base = BERT_base.predict([test_input_ids,test_attention_masks])